In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !python3 -m pip install tensorflow

In [2]:
# 코드 작성일 : 24.7.4 -> 대표메뉴
import os
from pykospacing import Spacing
from concurrent.futures import ThreadPoolExecutor

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
exec(open('setup/default.py').read())

/Users/user/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [5]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    return text

# 데이터프레임에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)



In [6]:
first_menu_list = menu_w_sold_out.groupby(['first_menu'],dropna=False).size().to_frame('size').reset_index().sort_values(by='size',ascending=False)

In [7]:
# 띄어쓰기
spacing = Spacing()
sentences = first_menu_list.first_menu

def process_spacing(sentence):
    return spacing(sentence)

# ThreadPoolExecutor를 사용하여 동시에 5개의 작업을 수행합니다.
with ThreadPoolExecutor(max_workers=5) as executor:
    spaced_sentences = list(executor.map(process_spacing, sentences))

spaced_menu = pd.DataFrame({
    'first_menu': sentences,
    'first_menu_sp': spaced_sentences
})

In [8]:
#spaced_menu.to_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv',index=False)
spaced_menu=pd.read_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv')

In [9]:
menu_w_sold_out=pd.merge(menu_w_sold_out,spaced_menu, how='left',on='first_menu') 

In [10]:
first_menu_list = menu_w_sold_out.first_menu_sp.value_counts().to_frame().reset_index()

In [11]:
first_menu_list.sort_values(by='count',ascending=False).iloc[:10,:]

,first_menu_sp,count
0,뼈 없는 감자탕,21
1,닭곰탕,18
2,콩비지찌개,17
3,돼지국밥,16
4,마파 두부 덮밥,16
5,돼지 양념구이,15
6,설렁탕,15
7,가츠 동 돈부리,15
8,통마늘 돼 지불백,15
9,옛날 카레라이스,15


In [17]:
# 비교할 텍스트
target_text = '마라탕'

# 유사도 계산하여 가장 유사한 메뉴 찾기
first_menu_list['similarity'] = first_menu_list['first_menu_sp'].apply(lambda x: fuzz.token_sort_ratio(x, target_text))
first_menu_list.sort_values(by='similarity',ascending=False).iloc[:10,:]

,first_menu_sp,count,similarity
111,마라 탕면,4,75
584,로제 마라 떡볶이,1,33
6,설렁탕,15,33
1,닭곰탕,18,33
12,양지탕,14,33
283,꽃도리탕,1,29
440,돈마호크,1,29
442,닭도리탕,1,29
413,동태 알탕,1,25
130,오므라이스,4,25
